# Environment Setup

In [ ]:
# Installing libraries
!pip install seaborn numpy pandas matplotlib nltk

In [45]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Error loading punkt: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>


KeyboardInterrupt: 

In [31]:
# pandas for maniplating, cleaning, and exploring data
import pandas as pd
# numerical computing in Python. arrays and performance optimization
import numpy as np
# plotting
import matplotlib.pyplot as plt
# built ontop of plt, heatmaps, boxplots, distribution plots
import seaborn as sns
# basic natural language processing. tokenizations
# stopwords, stemming, etc
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

## Split Datasets

In [15]:
train = pd.read_csv('/kaggle/input/2024-u-s-election-sentiment-on-x/train.csv')
test = pd.read_csv('/kaggle/input/2024-u-s-election-sentiment-on-x/test.csv')
val = pd.read_csv('/kaggle/input/2024-u-s-election-sentiment-on-x/val.csv')

# notes from LB 
##  The machine learning workflow: 
<br> 1) Obtain Data 
<br> 2) Exploratory data analysis (Look how many missing values (in data rows) you have, look for outliers in length, do TF-IDF to see which words are most important to your dataset, make lots of charts matplotlib, seaborn) 
<br> 3) clean data (pandas) 
<br> 4) split data (80/20 train/test) 
<br> 5) model.train() 
<br> 6) model.test
<br> 7) SKLearn accuracy metrics
<br> 8 ) chart results


# Exploratory Data Analysis

### Training Data
Training data from the 2024 US Presidential election, sourcced from X (fromerly Twitter). It includes post text, candidate information, party affiliation, sentitment labels, retweet counts, and like. This data is ideal for training machine learning models to analyze public sentiment and trends.

In [16]:
train.head()

,tweet_id,user_handle,timestamp,tweet_text,candidate,party,retweets,likes,sentiment
0,1,@user123,2024-11-03 08:45:00,Excited to see Kamala Harris leading the Democ...,Kamala Harris,Democratic Party,120,450,positive
1,2,@politicsFan,2024-11-03 09:15:23,Donald Trump's policies are the best for our e...,Donald Trump,Republican Party,85,300,positive
2,3,@greenAdvocate,2024-11-03 10:05:45,Jill Stein's environmental plans are exactly w...,Jill Stein,Green Party,60,200,positive
3,4,@indieVoice,2024-11-03 11:20:10,Robert Kennedy offers a fresh perspective outs...,Robert Kennedy,Independent,40,150,neutral
4,5,@libertyLover,2024-11-03 12:35:55,Chase Oliver's libertarian stance promotes tru...,Chase Oliver,Libertarian Party,30,120,positive


### Testing Data
This file provides testing data from the 2024 US Presidential election. It contains post text, candidate details, party affilication, and engagement metrics. This dataset is used to evaluate the performance of models trains on the training data.

In [17]:
test.head()

,tweet_id,user_handle,timestamp,tweet_text,candidate,party,retweets,likes,sentiment
0,501,@user1068,2025-02-11 08:30:10,Kamala Harris's new policy on education reform...,Kamala Harris,Democratic Party,85,320,positive
1,502,@politicsNew2,2025-02-11 09:15:20,Donald Trump's latest speech on the economy wa...,Donald Trump,Republican Party,100,450,positive
2,503,@greenVisionary2,2025-02-11 10:05:35,Jill Stein's environmental policies are a step...,Jill Stein,Green Party,60,200,positive
3,504,@indieWatcher2,2025-02-11 11:25:40,Robert Kennedy's independent run brings fresh ...,Robert Kennedy,Independent,30,120,neutral
4,505,@libertyFan4,2025-02-11 12:40:50,Chase Oliver's stance on gun rights is unwaver...,Chase Oliver,Libertarian Party,50,160,positive


### Validation Data
This file contains validation data for the 2024 US Presidential election. It includes post content, candidates, party affiliation, and sentiment labels. Use this file to fine-tune and validate model accuracy and performance. 

In [18]:
val.head()

,tweet_id,user_handle,timestamp,tweet_text,candidate,party,retweets,likes,sentiment
0,551,@user1077,2025-02-21 08:30:10,Kamala Harris's new policy on education reform...,Kamala Harris,Democratic Party,85,320,positive
1,552,@politicsNew3,2025-02-21 09:15:20,Donald Trump's latest speech on the economy wa...,Donald Trump,Republican Party,100,450,positive
2,553,@greenVisionary3,2025-02-21 10:05:35,Jill Stein's environmental policies are a step...,Jill Stein,Green Party,60,200,positive
3,554,@indieWatcher3,2025-02-21 11:25:40,Robert Kennedy's independent run brings fresh ...,Robert Kennedy,Independent,30,120,neutral
4,555,@libertyFan5,2025-02-21 12:40:50,Chase Oliver's stance on gun rights is unwaver...,Chase Oliver,Libertarian Party,50,160,positive


## `.info()` and `.value_counts()`

In [34]:
# stats summary of train data
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   tweet_id     500 non-null    int64 
 1   user_handle  500 non-null    object
 2   timestamp    500 non-null    object
 3   tweet_text   500 non-null    object
 4   candidate    500 non-null    object
 5   party        500 non-null    object
 6   retweets     500 non-null    int64 
 7   likes        500 non-null    int64 
 8   sentiment    500 non-null    object
dtypes: int64(3), object(6)
memory usage: 35.3+ KB


In [24]:
# test stats
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   tweet_id     50 non-null     int64 
 1   user_handle  50 non-null     object
 2   timestamp    50 non-null     object
 3   tweet_text   50 non-null     object
 4   candidate    50 non-null     object
 5   party        50 non-null     object
 6   retweets     50 non-null     int64 
 7   likes        50 non-null     int64 
 8   sentiment    50 non-null     object
dtypes: int64(3), object(6)
memory usage: 3.6+ KB


In [25]:
# val stats
val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   tweet_id     50 non-null     int64 
 1   user_handle  50 non-null     object
 2   timestamp    50 non-null     object
 3   tweet_text   50 non-null     object
 4   candidate    50 non-null     object
 5   party        50 non-null     object
 6   retweets     50 non-null     int64 
 7   likes        50 non-null     int64 
 8   sentiment    50 non-null     object
dtypes: int64(3), object(6)
memory usage: 3.6+ KB


### looking for how many missing values if have any duplicates

In [40]:
print('Duplicates in Train: ', train.duplicated().sum())

Duplicates in Train:  0


In [42]:
print('Duplicates in Test: ', test.duplicated().sum())

Duplicates in Train:  0


In [43]:
print('Duplicates in Val: ', val.duplicated().sum())

Duplicates in Val:  0


### describing numerical features

In [44]:
train.describe()

,tweet_id,retweets,likes
count,500.000000,500.000000,500.000000
mean,250.500000,64.236000,248.920000
std,144.481833,27.715349,120.562034
min,1.000000,18.000000,85.000000
25%,125.750000,40.000000,150.000000
50%,250.500000,70.000000,225.000000
75%,375.250000,90.000000,340.000000
max,500.000000,120.000000,500.000000


# Do TF-IDF to see most important words 

# 

Dataset overview for all 3 files

### `pandas.info()`
`.info()` is a pandas method that "prints a concise summary oof a DataFrame.
This method provides a concise summary about a DataFrame including data types, non-null values, and memeory usage for each column. <br>


In [20]:
print('Train Dataset Info:')
train.info()

Train Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   tweet_id     500 non-null    int64 
 1   user_handle  500 non-null    object
 2   timestamp    500 non-null    object
 3   tweet_text   500 non-null    object
 4   candidate    500 non-null    object
 5   party        500 non-null    object
 6   retweets     500 non-null    int64 
 7   likes        500 non-null    int64 
 8   sentiment    500 non-null    object
dtypes: int64(3), object(6)
memory usage: 35.3+ KB


In [21]:
print('Test Dataset Info: ')
test.info()

Test Dataset Info: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   tweet_id     50 non-null     int64 
 1   user_handle  50 non-null     object
 2   timestamp    50 non-null     object
 3   tweet_text   50 non-null     object
 4   candidate    50 non-null     object
 5   party        50 non-null     object
 6   retweets     50 non-null     int64 
 7   likes        50 non-null     int64 
 8   sentiment    50 non-null     object
dtypes: int64(3), object(6)
memory usage: 3.6+ KB


In [22]:
print('Validation Dataset Info: ')
val.info()

Validation Dataset Info: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   tweet_id     50 non-null     int64 
 1   user_handle  50 non-null     object
 2   timestamp    50 non-null     object
 3   tweet_text   50 non-null     object
 4   candidate    50 non-null     object
 5   party        50 non-null     object
 6   retweets     50 non-null     int64 
 7   likes        50 non-null     int64 
 8   sentiment    50 non-null     object
dtypes: int64(3), object(6)
memory usage: 3.6+ KB
